In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/refine-epitope-deep-learning')

Mounted at /content/drive/


In [2]:
!pip install transformers
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 9.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.3 MB/s 
     |████████████████████████████████| 596 kB 68.3 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 7.4 MB/s 
     |████████████████████████████████| 81 kB 9.9 MB/s 
     |████████████████████████████████| 209 kB 68.5 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 146 kB 76.3 MB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 
     |████████████████████████████████| 112 kB 75.2 MB/s 
  Created wheel for pyp

# Preprocess data 

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from transformers import EarlyStoppingCallback

In [4]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [5]:
def preprocess_data(data):

    # Preprocess data
    X = list(data["sequence"])
    y = list(data["label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

    train_dataset = Dataset(X_train_tokenized, y_train)
    val_dataset = Dataset(X_val_tokenized, y_val)
    return train_dataset, val_dataset

In [ ]:
df_train = pd.read_csv("./input/data_train.csv")

sequence_formatted = []
for seq in df_train['sequence'].values:
  sequence_formatted.append(" ".join(seq))

data = pd.DataFrame({'sequence':sequence_formatted, 'label':df_train['label'].tolist()})

#data = df_train

data_op = data[:int(len(data)/5)]


# Define pretrained tokenizer and model
batch_size=4
model_name = "Rostlab/prot_bert_bfd"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)



train_dataset_op, val_dataset_op = preprocess_data(data_op)
train_dataset, val_dataset = preprocess_data(data)

# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    f"{model_name}-finetuned-classification",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    optim="adamw_torch"
)

def model_init():
    return model

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset_op,
    eval_dataset=val_dataset_op,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


best_run = trainer.hyperparameter_search(n_trials=3, direction="maximize")



Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

...

***** Running Evaluation *****
  Num examples = 33
  Batch size = 4
Saving model checkpoint to Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-33
Configuration saved in Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-33/config.json
Model weights saved in Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-33/pytorch_model.bin
tokenizer config file saved in Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-33/tokenizer_config.json
Special tokens file saved in Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-33/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 33
  Batch size = 4
Saving model checkpoint to Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-66
Configuration saved in Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-66/config.json
Model weights saved in Rostlab/prot_bert_bfd-finetuned-classification/run-0/checkpoint-66/pytorch_model.bin
tokenizer config

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.601423,0.727273,0.600000,0.750000,0.666667


***** Running Evaluation *****
  Num examples = 33
  Batch size = 4
Saving model checkpoint to Rostlab/prot_bert_bfd-finetuned-classification/run-1/checkpoint-33
Configuration saved in Rostlab/prot_bert_bfd-finetuned-classification/run-1/checkpoint-33/config.json
Model weights saved in Rostlab/prot_bert_bfd-finetuned-classification/run-1/checkpoint-33/pytorch_model.bin
tokenizer config file saved in Rostlab/prot_bert_bfd-finetuned-classification/run-1/checkpoint-33/tokenizer_config.json
Special tokens file saved in Rostlab/prot_bert_bfd-finetuned-classification/run-1/checkpoint-33/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from Rostlab/prot_bert_bfd-finetuned-classification/run-1/checkpoint-33 (score: 0.6666666666666665).
[I 2022-06-22 01:13:53,527] Trial 1 finished with value: 2.7439393939393937 and parameters: {'learning_rate': 1.5059233155046064e-06, 'num_train_epochs': 1, 'seed': 5, 'per_device_tr

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.563004,0.757576,0.642857,0.750000,0.692308
2,No log,0.497265,0.787879,0.727273,0.666667,0.695652
3,No log,0.487600,0.818182,0.800000,0.666667,0.727273
4,No log,0.458207,0.848485,0.888889,0.666667,0.761905
5,No log,0.456902,0.848485,0.888889,0.666667,0.761905


***** Running Evaluation *****
  Num examples = 33
  Batch size = 4
Saving model checkpoint to Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-33
Configuration saved in Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-33/config.json
Model weights saved in Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-33/pytorch_model.bin
tokenizer config file saved in Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-33/tokenizer_config.json
Special tokens file saved in Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-33/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 33
  Batch size = 4
Saving model checkpoint to Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-66
Configuration saved in Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-66/config.json
Model weights saved in Rostlab/prot_bert_bfd-finetuned-classification/run-2/checkpoint-66/pytorch_model.bin
tokenizer config

## Set the model with the best parameters and run it on the full dataset

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train_dataset=train_dataset
trainer.eval_dataset=val_dataset

trainer.train()

***** Running training *****
  Num examples = 660
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 825


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.386974,0.855422,0.870370,0.734375,0.796610
2,No log,0.422713,0.855422,0.870370,0.734375,0.796610
3,No log,0.391086,0.879518,0.892857,0.781250,0.833333
4,0.362900,0.395698,0.879518,0.879310,0.796875,0.836066
5,0.362900,0.400908,0.879518,0.892857,0.781250,0.833333


***** Running Evaluation *****
  Num examples = 166
  Batch size = 4
Saving model checkpoint to Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-165
Configuration saved in Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-165/config.json
Model weights saved in Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-165/pytorch_model.bin
tokenizer config file saved in Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-165/tokenizer_config.json
Special tokens file saved in Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-165/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 166
  Batch size = 4
Saving model checkpoint to Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-330
Configuration saved in Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-330/config.json
Model weights saved in Rostlab/prot_bert_bfd-finetuned-classification/checkpoint-330/pytorch_model.bin
tokenizer config file saved in Rostlab/prot_bert_bfd-f

TrainOutput(global_step=825, training_loss=0.3157784016927083, metrics={'train_runtime': 829.4343, 'train_samples_per_second': 3.979, 'train_steps_per_second': 0.995, 'total_flos': 3278819860786800.0, 'train_loss': 0.3157784016927083, 'epoch': 5.0})

In [ ]:
# ----- 3. Predict -----#
# Load test data
#test_data = pd.read_csv("test.csv")
test = pd.read_csv("./input/data_test.csv")

sequence_formatted = []
for seq in test['sequence'].values:
  sequence_formatted.append(" ".join(seq))

test_data = pd.DataFrame({'sequence':sequence_formatted, 'label':test['label'].tolist()})


X_test = list(test_data["sequence"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)


***** Running Prediction *****
  Num examples = 207
  Batch size = 4


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

print("ROC_AUC:", roc_auc_score(test_data['label'], y_pred))

print(classification_report(test_data['label'], y_pred))

ROC_AUC: 0.8245798319327731
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       119
           1       0.85      0.75      0.80        88

    accuracy                           0.84       207
   macro avg       0.84      0.82      0.83       207
weighted avg       0.84      0.84      0.83       207

